In [1]:
import tensorflow as tf
import numpy as np
import pickle
from os import path
from datetime import datetime
from models import *

In [2]:
dataset_dir = "../data/GCN_datasets"
dataset_name = "case39_38000_2020.12.01-01:19:28" + ".pickle"
dataset_path = path.join(dataset_dir, dataset_name)
infile = open(dataset_path, 'rb')
dataset = pickle.load(infile)
infile.close()

Xs = dataset["feature"]
Ys = dataset["label"]

[my TensorFlow cheat-sheet](https://github.com/jhyun0919/deep_dive_into_tensorflow/blob/master/tutorials/TF_Basic/basic%20structure.ipynb)

# 0. Hyper-Parameters 

In [3]:
train_ratio = float(320 / 380)
test_ratio = float(30 / 380)

In [4]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
EPOCHS = 10

In [5]:
MODEL_ARCHI = "FCN1"
# MODEL_ARCHI = "FCN2"
# MODEL_ARCHI = "GCN"

# 1. Data pre-process

In [6]:
# scaling
Xs, X_scaler = data_scaler(Xs)
Ys, Y_scaler = data_scaler(Ys)

In [7]:
train_examples, val_examples, test_examples = split_dataset(Xs, 
                                                            train_ratio, 
                                                            test_ratio)
train_labels, val_labels, test_labels = split_dataset(Ys, 
                                                      train_ratio,
                                                      test_ratio)

In [8]:
print("> data shape (feature) / (label)")
print("  - trn:\t{} / {}".format(train_examples.shape, train_labels.shape))
print("  - val:\t{} / {}".format(val_examples.shape, val_labels.shape))
print("  - test:\t{} / {}".format(test_examples.shape, test_labels.shape))

> data shape (feature) / (label)
  - trn:	(32000, 88) / (32000, 46)
  - val:	(2999, 88) / (2999, 46)
  - test:	(3000, 88) / (3000, 46)


# 2. Data Feeder

- Ref. https://www.tensorflow.org/tutorials/load_data/numpy

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, 
                                                    train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_examples, 
                                                  val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, 
                                                   test_labels))

In [10]:
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

---

# 3. Architectures
## 3.1. FCN

## 3.2. GCN

# 4. Callbacks
## 4.1. checkpoint

In [41]:
case_name = "_".join(dataset_name.split(".")[0].split("_")[:-1])

checkpoint_dir = path.join('checkpoints', MODEL_ARCHI)
checkpoint_path = path.join(
    checkpoint_dir,
    case_name) + datetime.now().strftime("%Y%m%d-%H%M%S") + '.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                verbose=1)

## 4.2. tensorboard

In [42]:
%load_ext tensorboard
# Clear any logs from previous runs
# !rm -rf ./logs/

log_dir = path.join("logs/fit", MODEL_ARCHI)
log_dir = path.join(log_dir,
                    case_name) + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## 4.3. reduce learning rate

In [43]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1,
                                                 patience=3,
                                                 min_lr=0.00001)

# 5. Optimizer, Loss, and Metrics

In [44]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.MeanSquaredError()
metric_acc = tf.keras.metrics.KLDivergence()

# 6. Compile and Fit

In [45]:
model.compile(optimizer, loss, metrics=[metric_acc])

NameError: name 'model' is not defined

In [46]:
model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=EPOCHS,
    callbacks=[checkpoint, reduce_lr, tensorboard_callback],
)

NameError: name 'model' is not defined

# 7. Results
## 7.1. check training process via tensorboard

In [47]:
%tensorboard --logdir logs/gradient_tape

## 7.2. load the best weights and test the model

In [ ]:
model.load_weights(checkpoint_path)
model.evaluate(x_test, y_test)